# Cinetel Data Cleaning

Cinetel data scraped daily from their boxoffice published everyday at this [link](https://www.cinetel.it/pages/boxoffice.php?edperiodo=aWVyaQ==).

## Reading In Files

In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv(r"E:\data_analysis_python\cinetel\cinetel_.csv")
df1.head()

,Pos.,Titolo,Prima Progr.,Nazione,Distribuzione,Incasso,Presenze,Incasso al 16/11/2023,Presenze al 16/11/2023,2023-11-17 15:31:50.351708
0,1,C'E' ANCORA DOMANI,'2023-10-26'26/10/2023,ITA,VISION DISTRIBUTION,449991.06,66700,15231636.85,2202169,2023-11-17 15:31:50.351708
1,2,HUNGER GAMES - LA BALLATA DELL'USIGNOLO E DEL ...,'2023-11-15'15/11/2023,USA,MEDUSA FILM S.P.A.,238466.31,32469,584902.45,79311,2023-11-17 15:31:50.351708
2,3,THE MARVELS,'2023-11-08'08/11/2023,USA,WALT DISNEY S.M.P. ITALIA,60914.35,8628,2260142.44,305291,2023-11-17 15:31:50.351708
3,4,THANKSGIVING,'2023-11-16'16/11/2023,USA,EAGLE PICTURES S.P.A.,41232.57,5709,41232.57,5709,2023-11-17 15:31:50.351708
4,5,THE OLD OAK,'2023-11-16'16/11/2023,FRA,LUCKY RED DISTRIB.,36605.89,5879,56540.79,8609,2023-11-17 15:31:50.351708


---

In [3]:
df1 = pd.DataFrame(df1)

## Cleaning the dataframe

To clean the dataframe I decided to drop duplicate values and columns that aren't relevant to my analysis, such as `df1['Incasso al 16/11/2023']` and `df1['Presenze al 16/11/2023]`. I then decided to rename the existing columns using the snake_case rule in order to make the analysis easier.

In [4]:
df1 = df1.drop_duplicates()

In [5]:
# df1 = df1.drop(columns= ['Incasso al 16/11/2023', 'Presenze al 16/11/2023'])

# at first i thought these columns weren't relevant
# making visualizations, i realized i need the data to have accurate insights on movies that started their screenings before i started to scrape the data

In [6]:
df1 = df1.rename(columns = {'Pos.':'daily_rank', 'Titolo' : 'title', 'Prima Progr.' : 'first_screening_date', 'Nazione' : 'nation', 'Distribuzione' : 'distribution', 'Incasso' : 'daily_takings', 'Presenze' : 'daily_attendance','Incasso al 16/11/2023' : 'total_takings',  'Presenze al 16/11/2023' : 'total_attendance', '2023-11-17 15:31:50.351708' : 'date_pulled'})

In [7]:
# dropping the extra 'header row' that is imported everyday when the .csv file gets updated
df1 = df1[df1['daily_rank'].str.contains('Pos.')==False]

In [8]:
# making the 'title' and 'distribution' entries consistent by appling the title format
df1['title'] = df1['title'].apply(lambda x: x.title())
df1['distribution'] = df1['distribution'].apply(lambda x: x.title())

In [9]:
# cleaning the 'first_screening_date' entries
df1['first_screening_date'] = df1['first_screening_date'].str[12:]

In [10]:
# cleaning the column showing the date the data was pulled from the site
df1['date_pulled'] = pd.to_datetime(df1['date_pulled'])
df1['date_pulled'] = pd.to_datetime(df1['date_pulled']).dt.date

#extra tries:
#df1['date_pulled'] = df1['date_pulled'].str[:10]
#df1['date_pulled'] = df1['date_pulled'].str.replace('[^a-zA-Z0-9]', '', regex = True)
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: str(x))
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: x[0:2] + '/' + x[2:4] + '/' + x[4:8])

In [11]:
from datetime import timedelta, datetime

In [12]:
df1['first_screening_date']

0      26/10/2023
1      15/11/2023
2      08/11/2023
3      16/11/2023
4      16/11/2023
          ...    
599    28/12/2023
600    26/10/2023
601    04/01/2024
602    20/12/2023
603    04/01/2024
Name: first_screening_date, Length: 550, dtype: object

In [13]:
# formatting
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date'], errors='coerce')
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date']).dt.date

C:\Users\Franci\AppData\Local\Temp\ipykernel_35508\2741308956.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date'], errors='coerce')


In [14]:
df1['daily_takings'] = df1['daily_takings'].astype(float)
df1['daily_attendance'] = df1['daily_attendance'].astype(int)
df1['daily_rank'] = df1['daily_rank'].astype(int)
df1['total_takings'] = df1['total_takings'].astype(float)
df1['total_attendance'] = df1['total_attendance'].astype(int)

---

## Data manipulation

I decided to create new columns to get information I may need later on during the analysis.

In [15]:
# calculating the number of days of screenings for each movie 
df1['screening_days'] = datetime.today().date() - df1['first_screening_date']
df1['screening_days'] = df1['screening_days'].map(lambda x: str(x)[:-14])

In [16]:
df1['screening_days']

0      74
1      54
2      61
3      53
4      53
       ..
599    11
600    74
601     4
602    19
603     4
Name: screening_days, Length: 550, dtype: object

In [17]:
#df1['screening_days'] = df1['screening_days'].str.strip()
df1['screening_days'] = df1['screening_days'].replace('', '0')
df1['screening_days'] = df1['screening_days'].astype(int) + 1

In [18]:
# creating a column to show the date the 'daily_takings' and 'daily_attendace' refer to
df1['date'] = df1['date_pulled'] - timedelta(days=1)
df1['date'] = pd.to_datetime(df1['date'])

In [19]:
# creating a column showing the day of the week. it'll be useful for further analysis
df1['day_of_week'] = df1['date'].dt.day_name()

In [20]:
df1['avg_ticket_price'] = df1['daily_takings'] / df1['daily_attendance']

In [21]:
pd.set_option('display.max_rows', 8)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df1

,daily_rank,title,first_screening_date,nation,distribution,daily_takings,daily_attendance,total_takings,total_attendance,date_pulled,screening_days,date,day_of_week,avg_ticket_price
0,1,C'E' Ancora Domani,2023-10-26,ITA,Vision Distribution,449991.06,66700,15231636.85,2202169,2023-11-17,75,2023-11-16,Thursday,6.75
1,2,Hunger Games - La Ballata Dell'Usignolo E Del ...,2023-11-15,USA,Medusa Film S.P.A.,238466.31,32469,584902.45,79311,2023-11-17,55,2023-11-16,Thursday,7.34
2,3,The Marvels,2023-11-08,USA,Walt Disney S.M.P. Italia,60914.35,8628,2260142.44,305291,2023-11-17,62,2023-11-16,Thursday,7.06
3,4,Thanksgiving,2023-11-16,USA,Eagle Pictures S.P.A.,41232.57,5709,41232.57,5709,2023-11-17,54,2023-11-16,Thursday,7.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,7,C'E' Ancora Domani,2023-10-26,ITA,Vision Distribution,247102.80,32788,34538203.11,5052428,2024-01-08,75,2024-01-07,Sunday,7.54
601,8,Perfect Days,2024-01-04,JPN,Lucky Red Distrib.,227669.99,32421,670727.98,95450,2024-01-08,5,2024-01-07,Sunday,7.02
602,9,Aquaman E Il Regno Perduto (Aquaman And The Lo...,2023-12-20,USA,Warner Bros Italia S.P.A.,192959.17,24789,5145783.59,660596,2024-01-08,20,2024-01-07,Sunday,7.78
603,10,Wonder - White Bird,2024-01-04,USA,Medusa Film S.P.A.,134352.21,19089,435659.59,61256,2024-01-08,5,2024-01-07,Sunday,7.04


---

## Export as .csv

In [22]:
import os

In [23]:
clean_file_name = 'cinetel_clean.csv'
clean_file_path = os.path.join(os.getcwd(), clean_file_name)

In [24]:
df1.to_csv(clean_file_path, index=False)
print(f"Il DataFrame è stato esportato con successo in {clean_file_path}.")

Il DataFrame è stato esportato con successo in e:\data_analysis_python\cinetel\cinetel_clean.csv.


---

## Data Exploration

In [25]:
import seaborn as sns
import matplotlib.pyplot as plt

In [26]:
pd.set_option('display.max_rows', 8)

In [27]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 550 entries, 0 to 603
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   daily_rank            550 non-null    int32         
 1   title                 550 non-null    object        
 2   first_screening_date  548 non-null    object        
 3   nation                550 non-null    object        
 4   distribution          550 non-null    object        
 5   daily_takings         550 non-null    float64       
 6   daily_attendance      550 non-null    int32         
 7   total_takings         550 non-null    float64       
 8   total_attendance      550 non-null    int32         
 9   date_pulled           550 non-null    object        
 10  screening_days        550 non-null    int32         
 11  date                  550 non-null    datetime64[ns]
 12  day_of_week           550 non-null    object        
 13  avg_ticket_price      550

In [28]:
df1.describe()

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,date,avg_ticket_price
count,550.00,550.00,550.00,550.00,550.00,550.00,550,550.00
mean,5.50,195313.54,26989.77,4785570.21,683559.12,42.06,2023-12-12 12:13:05.454545408,7.09
min,1.00,6272.70,864.00,11652.78,1470.00,1.00,2023-11-16 00:00:00,3.70
25%,3.00,36350.88,5257.75,458486.36,64669.00,26.00,2023-11-29 00:00:00,6.54
50%,5.50,105609.80,15012.00,1713417.40,249998.00,40.00,2023-12-13 00:00:00,7.05
75%,8.00,251847.86,35315.75,4383998.64,603828.00,55.00,2023-12-26 00:00:00,7.50
max,10.00,1544231.00,211764.00,34538203.11,5052428.00,124.00,2024-01-07 00:00:00,21.18
std,2.87,242127.94,32586.29,8271506.43,1211086.89,21.79,NaN,1.30


In [29]:
df1.isnull().sum()

daily_rank              0
title                   0
first_screening_date    2
nation                  0
                       ..
screening_days          0
date                    0
day_of_week             0
avg_ticket_price        0
Length: 14, dtype: int64

In [30]:
df1.nunique()

daily_rank               10
title                    50
first_screening_date     30
nation                    8
                       ... 
screening_days           31
date                     53
day_of_week               7
avg_ticket_price        530
Length: 14, dtype: int64

In [31]:
df1.corr(numeric_only=True)

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
daily_rank,1.00,-0.62,-0.64,-0.30,-0.29,0.03,-0.00
daily_takings,-0.62,1.00,1.00,0.25,0.24,-0.17,0.14
daily_attendance,-0.64,1.00,1.00,0.28,0.27,-0.15,0.10
total_takings,-0.30,0.25,0.28,1.00,1.00,0.50,-0.09
total_attendance,-0.29,0.24,0.27,1.00,1.00,0.50,-0.10
screening_days,0.03,-0.17,-0.15,0.50,0.50,1.00,-0.25
avg_ticket_price,-0.00,0.14,0.10,-0.09,-0.10,-0.25,1.00


In [32]:
df1.groupby('title').mean(numeric_only=True).sort_values(by = 'total_attendance', ascending= False)

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
title,,,,,,,
C'E' Ancora Domani,3.13,366873.89,53916.42,28169271.33,4124777.76,75.00,6.69
Wonka,1.74,566885.00,74817.74,7073292.07,941705.85,26.00,7.47
Io Capitano,8.43,15747.99,3713.14,4279651.65,747385.43,124.00,4.25
Napoleon,3.50,268725.93,35959.86,5468367.03,730972.00,47.00,7.19
...,...,...,...,...,...,...,...
Doc. Nelle Tue Mani - Stagione 3,7.00,21826.97,2384.00,35222.35,3855.50,21.00,9.16
Royal Opera House 2023/2024,7.00,19784.50,1916.00,35873.50,3496.00,111.00,10.33
Seventeen Tour Follow To Japan: Live Viewing,9.00,66089.87,3121.00,66089.87,3121.00,24.00,21.18
Casanova Operapop - Il Film,9.00,12093.47,1256.33,25578.93,2715.33,43.00,9.62


In [33]:
df2 = df1.groupby('title').mean(numeric_only=True).sort_values(by = 'total_attendance', ascending= False)
df2

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
title,,,,,,,
C'E' Ancora Domani,3.13,366873.89,53916.42,28169271.33,4124777.76,75.00,6.69
Wonka,1.74,566885.00,74817.74,7073292.07,941705.85,26.00,7.47
Io Capitano,8.43,15747.99,3713.14,4279651.65,747385.43,124.00,4.25
Napoleon,3.50,268725.93,35959.86,5468367.03,730972.00,47.00,7.19
...,...,...,...,...,...,...,...
Doc. Nelle Tue Mani - Stagione 3,7.00,21826.97,2384.00,35222.35,3855.50,21.00,9.16
Royal Opera House 2023/2024,7.00,19784.50,1916.00,35873.50,3496.00,111.00,10.33
Seventeen Tour Follow To Japan: Live Viewing,9.00,66089.87,3121.00,66089.87,3121.00,24.00,21.18
Casanova Operapop - Il Film,9.00,12093.47,1256.33,25578.93,2715.33,43.00,9.62
